In [1]:
import pandas as pd


df_scaled = pd.read_csv("../data/admission_data_standardized.csv")
df_scaled.head()      

,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit
0,1.762107,1.746971,0.798829,1.093864,1.167321,1.764818,0.909112,0.92
1,0.627656,-0.067635,0.798829,0.596653,1.167321,0.455151,0.909112,0.76
2,-0.070467,-0.562528,-0.076600,-0.397769,0.052933,-1.005631,0.909112,0.72
3,0.453126,0.427257,-0.076600,0.099442,-1.061454,0.119339,0.909112,0.80
4,-0.244998,-0.727492,-0.952029,-1.392191,-0.504260,-0.653029,-1.099975,0.65
